In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import copy
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
data = pd.read_csv('Dataset/TC3/train.csv')

In [4]:
data

,down_m_s2,right_m_s2,forward_m_s2
0,-9.827705,-0.002695,-0.146190
1,-9.799362,0.044593,-0.128741
2,-9.793747,0.016577,-0.117054
3,-9.787281,-0.003876,-0.142800
4,-9.818456,-0.033057,-0.093290
...,...,...,...
43542,-9.739140,-0.045190,-1.426029
43543,-9.676372,0.021534,-1.440146
43544,-9.683149,0.007309,-1.399659
43545,-9.682378,0.011090,-1.445912


In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df_max_scaled = data.copy()
columns_to_scale = ['down_m_s2', 'right_m_s2', 'forward_m_s2']

scaler = MinMaxScaler()
for column in columns_to_scale:
    df_max_scaled[column] = scaler.fit_transform(np.array(df_max_scaled[column]).reshape(-1, 1) )
    
df_max_scaled

,down_m_s2,right_m_s2,forward_m_s2
0,0.605132,0.575266,0.754676
1,0.609411,0.592876,0.762363
2,0.610259,0.582443,0.767512
3,0.611236,0.574826,0.756169
4,0.606528,0.563959,0.777981
...,...,...,...
43542,0.618505,0.559441,0.190832
43543,0.627982,0.584288,0.184613
43544,0.626959,0.578991,0.202450
43545,0.627075,0.580399,0.182073


In [6]:
sequence_size = 4
sequences = []

for i in range(len(data) - sequence_size + 1):
    sequence = data.iloc[i:i + sequence_size]
    sequences.append(sequence)

sequences_df = pd.concat(sequences, ignore_index=True)

sequences_df

,down_m_s2,right_m_s2,forward_m_s2
0,-9.827705,-0.002695,-0.146190
1,-9.799362,0.044593,-0.128741
2,-9.793747,0.016577,-0.117054
3,-9.787281,-0.003876,-0.142800
4,-9.799362,0.044593,-0.128741
...,...,...,...
174171,-9.682378,0.011090,-1.445912
174172,-9.676372,0.021534,-1.440146
174173,-9.683149,0.007309,-1.399659
174174,-9.682378,0.011090,-1.445912


In [7]:
x_train = sequences_df.values.tolist()
x_train

[[-9.82770538330078, -0.0026945807039737, -0.146190345287323],
 [-9.799362182617188, 0.0445928163826465, -0.128741443157196],
 [-9.793747425079346, 0.0165771730244159, -0.1170536987483501],
 [-9.787280559539797, -0.0038757030852139, -0.1428001224994659],
 [-9.799362182617188, 0.0445928163826465, -0.128741443157196],
 [-9.793747425079346, 0.0165771730244159, -0.1170536987483501],
 [-9.787280559539797, -0.0038757030852139, -0.1428001224994659],
 [-9.818455696105955, -0.0330570265650749, -0.0932901874184608],
 [-9.793747425079346, 0.0165771730244159, -0.1170536987483501],
 [-9.787280559539797, -0.0038757030852139, -0.1428001224994659],
 [-9.818455696105955, -0.0330570265650749, -0.0932901874184608],
 [-9.820038795471191, 0.0261377803981304, -0.1185307726264],
 [-9.787280559539797, -0.0038757030852139, -0.1428001224994659],
 [-9.818455696105955, -0.0330570265650749, -0.0932901874184608],
 [-9.820038795471191, 0.0261377803981304, -0.1185307726264],
 [-9.806636810302734, 0.0067134331911802, 

In [8]:
input_size = 3
dropout = 0.1
num_layers=1
hidden_size=64
num_epochs=20
learning_rate=0.01

In [9]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Determine device
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, bias=True, batch_first=True, dropout=dropout, bidirectional=True).to(self.device)  # Move LSTM to device
        self.fc_1 = nn.Linear(hidden_size * 2, 128).to(self.device)  # Move Linear layers to device
        self.relu1 = nn.ReLU()
        self.fc_2 = nn.Linear(128, input_size).to(self.device)
        self._init_weights()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(self.device)  # Move initial states to device
        c0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(self.device)

        out, _ = self.rnn(x, (h0))
        out = self.fc_1(out)
        out = self.relu1(out)
        out = self.fc_2(out)

        return out

    def _init_weights(self):
        for layer in [self.rnn, self.fc_1, self.fc_2]:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
                nn.init.zeros_(layer.bias)
            elif isinstance(layer, nn.RNN):
                for name, param in layer.named_parameters():
                    if 'weight_ih' in name or 'weight_hh' in name:
                        nn.init.kaiming_normal_(param.data, nonlinearity='relu')
                    elif 'bias' in name:
                        nn.init.zeros_(param.data)

In [10]:
model = Model(input_size, hidden_size, num_layers, dropout)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

C:\Users\Archi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [11]:
model.to(device)
model.train()

Model(
  (rnn): RNN(3, 64, batch_first=True, dropout=0.1, bidirectional=True)
  (fc_1): Linear(in_features=128, out_features=128, bias=True)
  (relu1): ReLU()
  (fc_2): Linear(in_features=128, out_features=3, bias=True)
)

In [12]:
predicted_vectors = []

last_index = len(x_train) - 4
num_sequences = last_index // 4

for epoch in range(num_epochs):
    predicted_vectors.clear()
    total_loss = 0
    
    for idx in range(0, last_index, 4):
        sequence = torch.tensor(x_train[idx: idx + 4]).to(device)  
        optimizer.zero_grad()

        out_forward = model(sequence)
        predicted_next_element_1 = out_forward[-1, :].view(-1, 3)
        target_next_element = torch.tensor(x_train[idx + 7]).view(-1, 3).to(device)

        loss_forward_1 = criterion(predicted_next_element_1, target_next_element)
        total_loss += loss_forward_1.item()
        
        loss_forward_1.backward()
        optimizer.step()

        if idx % 100 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Sequence {idx // 4 + 1}/{num_sequences}, Loss: {loss_forward_1.item()}")

        predicted_vectors.append(predicted_next_element_1.detach().cpu().numpy())

    # Calculate average loss for the epoch
    avg_loss = total_loss / num_sequences

   

Epoch 1/20, Sequence 1/43543, Loss: 34.002220153808594
Epoch 1/20, Sequence 26/43543, Loss: 1.1040185689926147
Epoch 1/20, Sequence 51/43543, Loss: 0.051678504794836044
Epoch 1/20, Sequence 76/43543, Loss: 0.0004468057013582438
Epoch 1/20, Sequence 101/43543, Loss: 0.0014442948158830404
Epoch 1/20, Sequence 126/43543, Loss: 0.0011970693012699485
Epoch 1/20, Sequence 151/43543, Loss: 0.0006272736354731023
Epoch 1/20, Sequence 176/43543, Loss: 0.0010046673705801368
Epoch 1/20, Sequence 201/43543, Loss: 0.0012104472843930125
Epoch 1/20, Sequence 226/43543, Loss: 0.00013987398415338248
Epoch 1/20, Sequence 251/43543, Loss: 0.0004916100879199803
Epoch 1/20, Sequence 276/43543, Loss: 0.0010711012873798609
Epoch 1/20, Sequence 301/43543, Loss: 0.0006911882665008307
Epoch 1/20, Sequence 326/43543, Loss: 0.0023927369620651007
Epoch 1/20, Sequence 351/43543, Loss: 0.0009834237862378359
Epoch 1/20, Sequence 376/43543, Loss: 0.001618649228475988
Epoch 1/20, Sequence 401/43543, Loss: 0.000675650022

Epoch 1/20, Sequence 3426/43543, Loss: 0.0017877600621432066
Epoch 1/20, Sequence 3451/43543, Loss: 0.0004710551002062857
Epoch 1/20, Sequence 3476/43543, Loss: 0.0013793687103316188
Epoch 1/20, Sequence 3501/43543, Loss: 0.0006036959821358323
Epoch 1/20, Sequence 3526/43543, Loss: 0.005742189474403858
Epoch 1/20, Sequence 3551/43543, Loss: 0.0014824389945715666
Epoch 1/20, Sequence 3576/43543, Loss: 0.0029121898114681244
Epoch 1/20, Sequence 3601/43543, Loss: 0.0026247254572808743
Epoch 1/20, Sequence 3626/43543, Loss: 0.001380106434226036
Epoch 1/20, Sequence 3651/43543, Loss: 0.005892711691558361
Epoch 1/20, Sequence 3676/43543, Loss: 0.0011020720703527331
Epoch 1/20, Sequence 3701/43543, Loss: 0.0005759908235631883
Epoch 1/20, Sequence 3726/43543, Loss: 0.0009066448546946049
Epoch 1/20, Sequence 3751/43543, Loss: 0.008120929822325706
Epoch 1/20, Sequence 3776/43543, Loss: 0.000133019988425076
Epoch 1/20, Sequence 3801/43543, Loss: 0.004301102366298437
Epoch 1/20, Sequence 3826/4354

Epoch 1/20, Sequence 6826/43543, Loss: 0.0009591422858648002
Epoch 1/20, Sequence 6851/43543, Loss: 0.006385456304997206
Epoch 1/20, Sequence 6876/43543, Loss: 0.00654190918430686
Epoch 1/20, Sequence 6901/43543, Loss: 0.0025953417643904686
Epoch 1/20, Sequence 6926/43543, Loss: 0.001893290551379323
Epoch 1/20, Sequence 6951/43543, Loss: 0.005306690465658903
Epoch 1/20, Sequence 6976/43543, Loss: 0.013023669831454754
Epoch 1/20, Sequence 7001/43543, Loss: 0.0031838833820074797
Epoch 1/20, Sequence 7026/43543, Loss: 0.004268018063157797
Epoch 1/20, Sequence 7051/43543, Loss: 0.011732987128198147
Epoch 1/20, Sequence 7076/43543, Loss: 0.0011272735428065062
Epoch 1/20, Sequence 7101/43543, Loss: 0.003524432424455881
Epoch 1/20, Sequence 7126/43543, Loss: 0.000742114323657006
Epoch 1/20, Sequence 7151/43543, Loss: 0.002973471535369754
Epoch 1/20, Sequence 7176/43543, Loss: 0.0011317457538098097
Epoch 1/20, Sequence 7201/43543, Loss: 0.002212804276496172
Epoch 1/20, Sequence 7226/43543, Los

Epoch 1/20, Sequence 10226/43543, Loss: 0.00406877975910902
Epoch 1/20, Sequence 10251/43543, Loss: 0.0012326487340033054
Epoch 1/20, Sequence 10276/43543, Loss: 0.002509322017431259
Epoch 1/20, Sequence 10301/43543, Loss: 0.0038240046706050634
Epoch 1/20, Sequence 10326/43543, Loss: 0.006707462482154369
Epoch 1/20, Sequence 10351/43543, Loss: 0.0010100516956299543
Epoch 1/20, Sequence 10376/43543, Loss: 0.0019052628194913268
Epoch 1/20, Sequence 10401/43543, Loss: 0.0019250026671215892
Epoch 1/20, Sequence 10426/43543, Loss: 0.0028764428570866585
Epoch 1/20, Sequence 10451/43543, Loss: 0.004882843233644962
Epoch 1/20, Sequence 10476/43543, Loss: 0.000987808802165091
Epoch 1/20, Sequence 10501/43543, Loss: 0.0006748773157596588
Epoch 1/20, Sequence 10526/43543, Loss: 0.00033603067276999354
Epoch 1/20, Sequence 10551/43543, Loss: 0.0015989651437848806
Epoch 1/20, Sequence 10576/43543, Loss: 0.0006612743018195033
Epoch 1/20, Sequence 10601/43543, Loss: 0.00018348083540331572
Epoch 1/20, 

Epoch 1/20, Sequence 13576/43543, Loss: 0.00034445105120539665
Epoch 1/20, Sequence 13601/43543, Loss: 0.0002094172959914431
Epoch 1/20, Sequence 13626/43543, Loss: 0.001851102220825851
Epoch 1/20, Sequence 13651/43543, Loss: 0.00037989544216543436
Epoch 1/20, Sequence 13676/43543, Loss: 0.00043735653162002563
Epoch 1/20, Sequence 13701/43543, Loss: 0.0017007298301905394
Epoch 1/20, Sequence 13726/43543, Loss: 0.00042420532554388046
Epoch 1/20, Sequence 13751/43543, Loss: 0.0013829254312440753
Epoch 1/20, Sequence 13776/43543, Loss: 0.0007624239078722894
Epoch 1/20, Sequence 13801/43543, Loss: 0.00368537288159132
Epoch 1/20, Sequence 13826/43543, Loss: 0.004157755058258772
Epoch 1/20, Sequence 13851/43543, Loss: 0.0017656044801697135
Epoch 1/20, Sequence 13876/43543, Loss: 0.0003495559503789991
Epoch 1/20, Sequence 13901/43543, Loss: 0.000298025319352746
Epoch 1/20, Sequence 13926/43543, Loss: 0.0015321190003305674
Epoch 1/20, Sequence 13951/43543, Loss: 0.000565132184419781
Epoch 1/20

...

Epoch 20/20, Sequence 27901/43543, Loss: 0.0007892975118011236
Epoch 20/20, Sequence 27926/43543, Loss: 0.00038262864109128714
Epoch 20/20, Sequence 27951/43543, Loss: 7.814664422767237e-05
Epoch 20/20, Sequence 27976/43543, Loss: 0.0003244883846491575
Epoch 20/20, Sequence 28001/43543, Loss: 0.000485934637254104
Epoch 20/20, Sequence 28026/43543, Loss: 0.0003158443723805249
Epoch 20/20, Sequence 28051/43543, Loss: 0.00028829806251451373
Epoch 20/20, Sequence 28076/43543, Loss: 0.00017383063095621765
Epoch 20/20, Sequence 28101/43543, Loss: 0.0003557955496944487
Epoch 20/20, Sequence 28126/43543, Loss: 0.00010035434388555586
Epoch 20/20, Sequence 28151/43543, Loss: 0.00011796514445450157
Epoch 20/20, Sequence 28176/43543, Loss: 0.0009346320293843746
Epoch 20/20, Sequence 28201/43543, Loss: 6.299313099589199e-05
Epoch 20/20, Sequence 28226/43543, Loss: 0.0002525749441701919
Epoch 20/20, Sequence 28251/43543, Loss: 0.0005380291258916259
Epoch 20/20, Sequence 28276/43543, Loss: 7.25213321

Epoch 20/20, Sequence 31176/43543, Loss: 0.0006837453693151474
Epoch 20/20, Sequence 31201/43543, Loss: 0.00025222363183274865
Epoch 20/20, Sequence 31226/43543, Loss: 0.0010104741668328643
Epoch 20/20, Sequence 31251/43543, Loss: 0.000288087350782007
Epoch 20/20, Sequence 31276/43543, Loss: 0.00390995666384697
Epoch 20/20, Sequence 31301/43543, Loss: 0.0023138965480029583
Epoch 20/20, Sequence 31326/43543, Loss: 0.00013723564916290343
Epoch 20/20, Sequence 31351/43543, Loss: 0.00024807220324873924
Epoch 20/20, Sequence 31376/43543, Loss: 0.000523448281455785
Epoch 20/20, Sequence 31401/43543, Loss: 1.8829150576493703e-05
Epoch 20/20, Sequence 31426/43543, Loss: 0.0036892779171466827
Epoch 20/20, Sequence 31451/43543, Loss: 0.00022363060270436108
Epoch 20/20, Sequence 31476/43543, Loss: 0.0014138509286567569
Epoch 20/20, Sequence 31501/43543, Loss: 0.0003365934535395354
Epoch 20/20, Sequence 31526/43543, Loss: 0.0005280068726278841
Epoch 20/20, Sequence 31551/43543, Loss: 0.00026809662

Epoch 20/20, Sequence 34426/43543, Loss: 0.0007345090853050351
Epoch 20/20, Sequence 34451/43543, Loss: 0.0008560833521187305
Epoch 20/20, Sequence 34476/43543, Loss: 0.00031560735078528523
Epoch 20/20, Sequence 34501/43543, Loss: 8.38393607409671e-05
Epoch 20/20, Sequence 34526/43543, Loss: 0.0001892429281724617
Epoch 20/20, Sequence 34551/43543, Loss: 0.0004214716318529099
Epoch 20/20, Sequence 34576/43543, Loss: 0.00039640924660488963
Epoch 20/20, Sequence 34601/43543, Loss: 2.3936072466312908e-05
Epoch 20/20, Sequence 34626/43543, Loss: 0.0006593847647309303
Epoch 20/20, Sequence 34651/43543, Loss: 6.998153548920527e-05
Epoch 20/20, Sequence 34676/43543, Loss: 0.0013055995805189013
Epoch 20/20, Sequence 34701/43543, Loss: 0.0003158763574901968
Epoch 20/20, Sequence 34726/43543, Loss: 0.0006992955459281802
Epoch 20/20, Sequence 34751/43543, Loss: 0.0003844382590614259
Epoch 20/20, Sequence 34776/43543, Loss: 0.011154230684041977
Epoch 20/20, Sequence 34801/43543, Loss: 0.00158713792

Epoch 20/20, Sequence 37726/43543, Loss: 0.0003364710137248039
Epoch 20/20, Sequence 37751/43543, Loss: 6.518852023873478e-05
Epoch 20/20, Sequence 37776/43543, Loss: 0.00012298038927838206
Epoch 20/20, Sequence 37801/43543, Loss: 0.0007139278459362686
Epoch 20/20, Sequence 37826/43543, Loss: 0.00067113246768713
Epoch 20/20, Sequence 37851/43543, Loss: 0.00024048527120612562
Epoch 20/20, Sequence 37876/43543, Loss: 0.0021007631439715624
Epoch 20/20, Sequence 37901/43543, Loss: 0.0006728784646838903
Epoch 20/20, Sequence 37926/43543, Loss: 0.0002312603173777461
Epoch 20/20, Sequence 37951/43543, Loss: 0.0005429871380329132
Epoch 20/20, Sequence 37976/43543, Loss: 7.811694376869127e-05
Epoch 20/20, Sequence 38001/43543, Loss: 0.0015485695330426097
Epoch 20/20, Sequence 38026/43543, Loss: 0.0005506098968908191
Epoch 20/20, Sequence 38051/43543, Loss: 0.027963219210505486
Epoch 20/20, Sequence 38076/43543, Loss: 0.007282329257577658
Epoch 20/20, Sequence 38101/43543, Loss: 0.01855830475687

Epoch 20/20, Sequence 41026/43543, Loss: 0.0005308621912263334
Epoch 20/20, Sequence 41051/43543, Loss: 0.0003199010679963976
Epoch 20/20, Sequence 41076/43543, Loss: 1.011305175779853e-05
Epoch 20/20, Sequence 41101/43543, Loss: 0.0004774953704327345
Epoch 20/20, Sequence 41126/43543, Loss: 0.0003537186421453953
Epoch 20/20, Sequence 41151/43543, Loss: 0.0010919796768575907
Epoch 20/20, Sequence 41176/43543, Loss: 0.001040074392221868
Epoch 20/20, Sequence 41201/43543, Loss: 0.0008996037649922073
Epoch 20/20, Sequence 41226/43543, Loss: 0.001572935376316309
Epoch 20/20, Sequence 41251/43543, Loss: 0.001531790941953659
Epoch 20/20, Sequence 41276/43543, Loss: 0.0010743787279352546
Epoch 20/20, Sequence 41301/43543, Loss: 0.00010241847485303879
Epoch 20/20, Sequence 41326/43543, Loss: 0.00013130626757629216
Epoch 20/20, Sequence 41351/43543, Loss: 0.0006436239928007126
Epoch 20/20, Sequence 41376/43543, Loss: 0.00019392475951462984
Epoch 20/20, Sequence 41401/43543, Loss: 0.000375021219

In [29]:
model_path = 'saved_model_RNN_Ardupilot.pth'

torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss_forward_1,
}, model_path)

print("Model saved successfully!")

Model saved successfully!


In [30]:
predicted_array = np.squeeze(np.array(predicted_vectors))

df_pred = pd.DataFrame(predicted_array, columns=['down_m_s2', 'right_m_s2', 'forward_m_s2'])

In [31]:
df_pred

,down_m_s2,right_m_s2,forward_m_s2
0,-9.691866,0.010951,-1.411167
1,-9.691614,0.014572,-1.352617
2,-9.689302,0.018766,-1.265962
3,-9.686094,0.021347,-1.162079
4,-9.684598,0.024189,-1.047446
...,...,...,...
43538,-9.675835,-0.024696,-1.400391
43539,-9.679916,-0.021581,-1.398169
43540,-9.683550,-0.014903,-1.399684
43541,-9.686853,-0.006890,-1.401057


In [32]:
num_sequences = last_index // 4 
y = torch.zeros(num_sequences)
y_hat = torch.zeros(num_sequences)

In [33]:
std_dev = data.std()
std_dev_array = np.array(std_dev).reshape(1, -1)
std_dev_array

array([[0.37416795, 0.14968165, 0.45189674]])

In [34]:
std_dev_array = np.array(std_dev).reshape(1, -1)
std_dev_array

array([[0.37416795, 0.14968165, 0.45189674]])

In [35]:
deviation_threshold = np.array(3 * std_dev).reshape(1, -1)
deviation_threshold
deviation_threshold_1=deviation_threshold[0][0]
deviation_threshold_2=deviation_threshold[0][1]
deviation_threshold_3=deviation_threshold[0][2]

In [36]:
for i in range(len(df_pred) - 4):
    if (abs(df_pred['down_m_s2'][i] - data['down_m_s2'][i + 4]) > deviation_threshold_1 or
        abs(df_pred['right_m_s2'][i] - data['right_m_s2'][i + 4]) > deviation_threshold_2 or
        abs(df_pred['forward_m_s2'][i] - data['forward_m_s2'][i + 4]) > deviation_threshold_3):
        y_hat[i]=1


In [37]:
y_hat

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [38]:
test_data_1=pd.read_csv('Dataset/TC3/test_TC3.csv')
test_data = test_data_1.iloc[:, :3]

In [39]:
test_data

,down_m_s2,right_m_s2,forward_m_s2
0,-9.712459,-0.000407,-1.412389
1,-9.713698,-0.037881,-1.406507
2,-9.678647,-0.012233,-1.419303
3,-9.662002,-0.015255,-1.452367
4,-9.710204,-0.007325,-1.392143
...,...,...,...
10882,-2.991808,1.743833,1.745898
10883,-15.280162,-1.321779,-1.311857
10884,-3.621236,1.574034,1.578042
10885,-14.645958,-1.113395,-1.165361


In [40]:
sequence_size = 4
sequences_test = []

for i in range(len(test_data) - sequence_size + 1):
    sequence_test = test_data.iloc[i:i + sequence_size]
    sequences_test.append(sequence_test)

sequences_test_df = pd.concat(sequences_test, ignore_index=True)

sequences_test_df

,down_m_s2,right_m_s2,forward_m_s2
0,-9.712459,-0.000407,-1.412389
1,-9.713698,-0.037881,-1.406507
2,-9.678647,-0.012233,-1.419303
3,-9.662002,-0.015255,-1.452367
4,-9.713698,-0.037881,-1.406507
...,...,...,...
43531,-14.645958,-1.113395,-1.165361
43532,-15.280162,-1.321779,-1.311857
43533,-3.621236,1.574034,1.578042
43534,-14.645958,-1.113395,-1.165361


In [42]:
model_path = 'saved_model_RNN_Ardupilot.pth'
checkpoint = torch.load(model_path)

In [43]:
model = Model(input_size, hidden_size, num_layers, dropout)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

C:\Users\Archi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Model(
  (rnn): RNN(3, 64, batch_first=True, dropout=0.1, bidirectional=True)
  (fc_1): Linear(in_features=128, out_features=128, bias=True)
  (relu1): ReLU()
  (fc_2): Linear(in_features=128, out_features=3, bias=True)
)

In [44]:
x_test = sequences_test_df.values.tolist()
x_test = torch.tensor(x_test).to(device)

In [45]:
with torch.no_grad():
    predicted_vectors_test = []
    for idx in range(0, len(x_test), sequence_size):
        sequence_test = x_test[idx:idx + sequence_size].to(device)  # Move input tensor to the same device as the model

        out_forward_test = model(sequence_test)
        predicted_next_element_test = out_forward_test[-1, :].view(-1, 3)
        predicted_vectors_test.append(predicted_next_element_test.cpu().numpy())


In [46]:
predicted_vectors_test

[array([[-9.694128  ,  0.01069596, -1.4117393 ]], dtype=float32),
 array([[-9.693905  ,  0.01069235, -1.4116912 ]], dtype=float32),
 array([[-9.6940975 ,  0.01069547, -1.4117329 ]], dtype=float32),
 array([[-9.694042  ,  0.01069459, -1.4117211 ]], dtype=float32),
 array([[-9.694006 ,  0.010694 , -1.4117132]], dtype=float32),
 array([[-9.693966  ,  0.01069334, -1.4117044 ]], dtype=float32),
 array([[-9.693928  ,  0.01069272, -1.4116962 ]], dtype=float32),
 array([[-9.69404   ,  0.01069455, -1.4117205 ]], dtype=float32),
 array([[-9.694138  ,  0.01069613, -1.4117416 ]], dtype=float32),
 array([[-9.693949  ,  0.01069307, -1.4117007 ]], dtype=float32),
 array([[-9.693985  ,  0.01069365, -1.4117085 ]], dtype=float32),
 array([[-9.693971  ,  0.01069342, -1.4117055 ]], dtype=float32),
 array([[-9.69409   ,  0.01069535, -1.4117311 ]], dtype=float32),
 array([[-9.6941395 ,  0.01069616, -1.411742  ]], dtype=float32),
 array([[-9.694096  ,  0.01069544, -1.4117323 ]], dtype=float32),
 array([[-9.6

In [54]:
predicted_test_array = np.squeeze(np.array(predicted_vectors_test))

df_test_pred = pd.DataFrame(predicted_test_array, columns=['down_m_s2', 'right_m_s2', 'forward_m_s2'])

In [55]:
df_test_pred

,down_m_s2,right_m_s2,forward_m_s2
0,-9.694128,0.010696,-1.411739
1,-9.693905,0.010692,-1.411691
2,-9.694098,0.010695,-1.411733
3,-9.694042,0.010695,-1.411721
4,-9.694006,0.010694,-1.411713
...,...,...,...
10879,-9.445044,0.006655,-1.357878
10880,-9.637032,0.009770,-1.399393
10881,-9.647753,0.009944,-1.401711
10882,-9.678429,0.010441,-1.408345


In [49]:
num_sequences_1 = len(sequences_test_df) // 4 
y_test = torch.zeros(num_sequences_1)
y_hat_test = torch.zeros(num_sequences_1)

In [50]:
std_dev_1 = test_data.std()
std_dev_array_1 = np.array(std_dev_1).reshape(1, -1)
std_dev_array_1

array([[1.59174962, 0.39215243, 0.66077279]])

In [51]:
deviation_threshold_test = np.array(3 * std_dev).reshape(1, -1)
deviation_threshold_test
deviation_threshold_test_1=deviation_threshold_test[0][0]
deviation_threshold_test_2=deviation_threshold_test[0][1]
deviation_threshold_test_3=deviation_threshold_test[0][2]

In [52]:
len(df_test_pred)

10884

In [56]:
for i in range(len(df_test_pred)):
    if (abs(df_pred['down_m_s2'][i] - data['down_m_s2'][i + 4]) > deviation_threshold_1 or
        abs(df_pred['right_m_s2'][i] - data['right_m_s2'][i + 4]) > deviation_threshold_2 or
        abs(df_pred['forward_m_s2'][i] - data['forward_m_s2'][i + 4]) > deviation_threshold_3):
        y_hat_test[i]=1


In [57]:
y_hat_test.shape

torch.Size([10884])

In [58]:
y_hat_test

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [59]:
test_data_1=pd.read_csv('Dataset/TC3/test_TC3.csv')
column_4_values = test_data_1.iloc[3:, 3].values

In [60]:
column_4_values
column_4_values.shape

(10884,)

In [61]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(column_4_values, y_hat_test)
print("Accuracy:", accuracy)


Accuracy: 0.9185042263873576
